In [1]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3


In [2]:
#path to image folder and load 3d model
path = r"3dTo2d\totalUncertancy"
img_paths = natsorted(glob(os.path.join(path, "*.png")))
dim = 735
box_dim = LR.BOX_DIM
flip = False
#load 3D model
path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(box_dim, path_real)

print(img_paths)


['3dTo2d\\totalUncertancy\\1_-87_+27_angle.png', '3dTo2d\\totalUncertancy\\2_+00_+00_angle.png', '3dTo2d\\totalUncertancy\\3_+87_+27_angle.png', '3dTo2d\\totalUncertancy\\4_+87_-27_angle.png', '3dTo2d\\totalUncertancy\\5_-75_-22_angle.png', '3dTo2d\\totalUncertancy\\6_-43_-11_angle.png', '3dTo2d\\totalUncertancy\\7_-43_+13_angle.png', '3dTo2d\\totalUncertancy\\8_+43_-13_angle.png', '3dTo2d\\totalUncertancy\\9_+43_+13_angle.png']


In [3]:
#read all images and angles
#format -> x_(+/-)**_(+/-)**_angle.png
prim_rots = []
secd_rots = []
imgs = []

for p in img_paths:
  img = cv2.imread(p, 0)
  f = os.path.basename(p)
  prim_a = int(f[2:5])
  secd_a = int(f[6:9])
  prim_rots.append(prim_a)
  secd_rots.append(secd_a)
  imgs.append(img)
  


In [4]:
#create back-projection for each image
boxes = []

for i in imgs:
  line_coords = LR.createCoordsForLoadedImageFrom3d(i, flip, dim)
  world_coords = LR.performBresenham3D(
      line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  boxes.append(box)
  print("produced box")


produced box
produced box
produced box
produced box
produced box
produced box
produced box
produced box
produced box


In [5]:
#add rotation to each box 
for i in range(len(boxes)):
  box_rot = LR.rotatePrimSecond(boxes[i], prim_rots[i], secd_rots[i])
  box_rot = np.where(box_rot > 0.5, 1, 0)
  boxes[i] = box_rot 
  print("Rotated box ", i)


Rotated box  0
Rotated box  1
Rotated box  2
Rotated box  3
Rotated box  4
Rotated box  5
Rotated box  6
Rotated box  7
Rotated box  8


In [6]:
# Save box projections for each image as numpy arrays
for i in range(len(boxes)):
  p = img_paths[i]
  f = os.path.basename(p)
  angle = f[2:9]
  np.save(r"model_uncertainty\normal\N" + angle, boxes[i])

In [7]:
"""#find uncertancy
uncertainty = []
prev_box_sum = boxes[0]
for b in boxes:
  #add box to previous box
  box_overlap= prev_box_sum + b
  box_overlap = np.where(box_overlap == 2, 1, 0)  # save overlap
  prev_box_sum = box_overlap

  #find overlap with original
  box_sum = box_overlap + real_box
  box_sum = np.where(box_sum == 2, 1, 0)  # save overlap
  uncertainty.append(np.sum(box_sum))
  print("box done")

print(uncertainty/np.sum(real_box))"""


'#find uncertancy\nuncertainty = []\nprev_box_sum = boxes[0]\nfor b in boxes:\n  #add box to previous box\n  box_overlap= prev_box_sum + b\n  box_overlap = np.where(box_overlap == 2, 1, 0)  # save overlap\n  prev_box_sum = box_overlap\n\n  #find overlap with original\n  box_sum = box_overlap + real_box\n  box_sum = np.where(box_sum == 2, 1, 0)  # save overlap\n  uncertainty.append(np.sum(box_sum))\n  print("box done")\n\nprint(uncertainty/np.sum(real_box))'